In [17]:
import cv2
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
import pandas as pd
from PIL import Image, ImageTk, ImageDraw
import pytesseract
import numpy as np
import tkinter.messagebox as messagebox


# Global variables to store extracted data
df = None
selected_box = None

# Global variables to store selected areas
column_one_area = None

# Create tkinter window
root = tk.Tk()
root.title("OCR Tool")

# Create label for displaying image
label_image = tk.Label(root)
label_image.pack()


def open_image():
    global image_tk, image_pil, label_image, column_one_area

    # Open a file dialog to select an image file
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")])

    # Open the selected image using PIL
    image_pil = Image.open(file_path)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_pil)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)

    # Reset the selected areas
    column_one_area = None


# Function to select column one area
def Select area():
    global column_one_area

    # Wait for left mouse button click event
    label_image.bind("<Button 1>", on_column_one_area_click)


def on_column_one_area_click(event):
    global column_one_area

    # Save the starting point of the selection
    column_one_area = (event.x, event.y)

    # Wait for left mouse button release event
    label_image.bind("<ButtonRelease 1>", on_column_one_area_release)


def on_column_one_area_release(event):
    global column_one_area

    # Save the ending point of the selection
    column_one_area += (event.x, event.y)

    # Update display
    update_display()


def update_display():
    global image_pil, column_one_area
    # Create a copy of the original PIL image
    image_display = image_pil.copy()

    # Draw rectangles for selected areas on the image
    if column_one_area is not None:
        draw_rectangle(image_display, column_one_area)

    # Convert the PIL image to a PhotoImage object
    image_tk = ImageTk.PhotoImage(image_display)

    # Update the label image with the PhotoImage
    label_image.config(image=image_tk)
    label_image.image = image_tk


# Function to draw a rectangle on an image
def draw_rectangle(image, area):
    draw = ImageDraw.Draw(image)
    draw.rectangle(area, outline='red', width=2)


def process_image():
    global df  # Use the global variables
    global image_pil, column_one_area
    import numpy as np

    column_one_image = image_pil.crop(column_one_area)

    
    # Crop the selected areas from the image
    img = np.array(column_one_image)
    

    # Convert the image to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Display the original image
    plt.imshow(img, cmap='gray')
    plt.title("Original Image")
    plt.show()

    # Apply global thresholding
    thresh, img_bin = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    img_bin = 255 - img_bin
    
    # Display the thresholded image
    plt.imshow(img_bin, cmap='gray')
    plt.title("Inverted Image with global thresholding")
    plt.show()

    
    img_bin1 = 255-img
    thresh1,img_bin1_otsu = cv2.threshold(img_bin1,128,255,cv2.THRESH_OTSU)
    plotting = plt.imshow(img_bin1_otsu,cmap='gray')
    plt.title("Inverted Image with otsu thresh holding")
    plt.show()
    img_bin2 = 255-img
    thresh1,img_bin_otsu = cv2.threshold(img_bin2,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    plotting = plt.imshow(img_bin_otsu,cmap='gray')
    plt.title("Inverted Image with otsu thresh holding")
    plt.show()
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    print(kernel)
    import numpy as np
    plt.figure(figsize= (30,30))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, np.array(img).shape[1]//100))
    eroded_image = cv2.erode(img_bin_otsu, vertical_kernel, iterations=3)
    plt.subplot(151),plt.imshow(eroded_image, cmap = 'gray')
    plt.title('Image after erosion with vertical kernel'), plt.xticks([]), plt.yticks([])
    vertical_lines = cv2.dilate(eroded_image, vertical_kernel, iterations=3)
    plt.subplot(152),plt.imshow(vertical_lines, cmap = 'gray')
    plt.title('Image after dilation with vertical kernel'), plt.xticks([]), plt.yticks([])
    plt.show()
    plt.figure(figsize= (30,30))
    hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (np.array(img).shape[1]//100, 1))
    horizontal_lines = cv2.erode(img_bin, hor_kernel, iterations=5)
    plt.subplot(153),plt.imshow(horizontal_lines, cmap = 'gray')
    plt.title('Image after erosion with horizontal kernel'), plt.xticks([]), plt.yticks([])
    horizontal_lines = cv2.dilate(horizontal_lines, hor_kernel, iterations=5)
    plt.subplot(154),plt.imshow(horizontal_lines, cmap = 'gray')
    plt.title('Image after dilation with horizontal kernel'), plt.xticks([]), plt.yticks([])
    plt.show()
    plt.figure(figsize= (30,30))
    vertical_horizontal_lines = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5, 0.0)
    vertical_horizontal_lines = cv2.erode(~vertical_horizontal_lines, kernel, iterations=3)
    plt.subplot(151),plt.imshow(vertical_horizontal_lines, cmap = 'gray')
    plt.title('Erosion'), plt.xticks([]), plt.yticks([])
    thresh, vertical_horizontal_lines = cv2.threshold(vertical_horizontal_lines,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    plt.subplot(152),plt.imshow(vertical_horizontal_lines, cmap = 'gray')
    plt.title('global and otsu thresholding'), plt.xticks([]), plt.yticks([])
    bitxor = cv2.bitwise_xor(img,vertical_horizontal_lines)
    plt.subplot(153),plt.imshow(bitxor, cmap = 'gray')
    plt.title('Horizontal and vertical lines image bitxor'), plt.xticks([]), plt.yticks([])
    bitnot = cv2.bitwise_not(bitxor)
    plt.subplot(154),plt.imshow(bitnot, cmap = 'gray')
    plt.title('Horizontal and vertical lines image with bitnot'), plt.xticks([]), plt.yticks([])
    plt.show()


    import pytesseract


    contours, hierarchy = cv2.findContours(vertical_horizontal_lines, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    boundingBoxes = [cv2.boundingRect(contour) for contour in contours]

    # Sort contours and bounding boxes based on y-coordinate of the bounding box

    (contours, boundingBoxes) = zip(*sorted(zip(contours, boundingBoxes),key=lambda x:x[1][1]))
    boxes = []
    for contour in contours:
      x, y, w, h = cv2.boundingRect(contour)
      if (w<1000 and h<500):
        image = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        boxes.append([x,y,w,h])
    plotting = plt.imshow(image,cmap='gray')
    plt.title("Identified contours")
    plt.show()

    rows=[]
    columns=[]
    heights = [boundingBoxes[i][3] for i in range(len(boundingBoxes))]
    mean = np.mean(heights)
    print(mean)
    columns.append(boxes[0])
    previous=boxes[0]

    # Group boxes into rows and columns based on y-coordinate

    for i in range(1,len(boxes)):
      if(boxes[i][1]<=previous[1]+mean/2):
        columns.append(boxes[i])
        previous=boxes[i]
        if(i==len(boxes)-1):
          rows.append(columns)
      else:
        rows.append(columns)
        columns=[]
        previous = boxes[i]
        columns.append(boxes[i])
    print("Rows")
    for row in rows:
      print(row)

    total_cells=0
    for i in range(len(row)):
      if len(row[i]) > total_cells:
        total_cells = len(row[i])
    print(total_cells)

    center = [int(rows[i][j][0]+rows[i][j][2]/2) for j in range(len(rows[i])) if rows[0]]
    print(center)
    center=np.array(center)
    center.sort()
    print(center)

    boxes_list = []
    for i in range(len(rows)):
        l=[]
        for k in range(total_cells):
            l.append([])
        for j in range(len(rows[i])):
            diff = abs(center-(rows[i][j][0]+rows[i][j][2]/4))
            minimum = min(diff)
            indexing = list(diff).index(minimum)
            l[indexing].append(rows[i][j])
        boxes_list.append(l)
    for box in boxes_list:
      print(box)

    dataframe_final=[]
    # Set the path to Tesseract executable
    pytesseract.pytesseract.tesseract_cmd = r'D:\machine_learning2023\tesseract.exe'

    for i in range(len(boxes_list)):
        for j in range(len(boxes_list[i])):
            s=''
            if(len(boxes_list[i][j])==0):
                dataframe_final.append(' ')
            else:
                for k in range(len(boxes_list[i][j])):
                    y,x,w,h = boxes_list[i][j][k][0],boxes_list[i][j][k][1], boxes_list[i][j][k][2],boxes_list[i][j][k][3]
                    roi = bitnot[x:x+h, y:y+w]
                    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1))
                    border = cv2.copyMakeBorder(roi,2,2,2,2, cv2.BORDER_CONSTANT,value=[255,255])
                    resizing = cv2.resize(border, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
                    dilation = cv2.dilate(resizing, kernel,iterations=1)
                    erosion = cv2.erode(dilation, kernel,iterations=2)                
                    out = pytesseract.image_to_string(erosion)
                    if(len(out)==0):
                        out = pytesseract.image_to_string(erosion)
                    s = s +" "+ out
                dataframe_final.append(s)
    print(dataframe_final)
    print('OCR completed')
      
    arr = np.array(dataframe_final)
    arr
    import pandas as pd
    dataframe = pd.DataFrame(arr.reshape(len(rows), total_cells))
    df = dataframe.style.set_properties(align="left")
    print(df)
    print(dataframe)
    messagebox.showinfo("OCR Completed", "OCR process has completed successfully.")
    
    
def save_excel():
    global df  # Use the global variable
    if df is not None:
        # Prompt the user to select a location to save the Excel file
        file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel Files", "*.xlsx")])
        if file_path:
            # Save the extracted data to the selected file location
            df.to_excel(file_path, index=False)
            print("Excel file saved at:", file_path)
        else:
            print("Save operation cancelled.")
    else:
        print("No data to save.")


button_open = tk.Button(root, text="Open Image", command=open_image)
button_open.pack()

button_select_column_one = tk.Button(root, text="Select Column One", command=Select area)
button_select_column_one.pack()

button_ocr = tk.Button(root, text="Perform OCR", command=process_image)
button_ocr.pack()



save_excel_button = tk.Button(root, text="Save Excel", command=save_excel)
save_excel_button.pack()



root.mainloop()
